In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, mean_absolute_error

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.feature_selection import SelectKBest
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
import numpy as np


In [2]:
df = pd.read_csv("diamonds_train.csv", index_col=0)

In [3]:
df["cut"] = df["cut"].apply(lambda x: x.replace("Fair","0"))
df["cut"] = df["cut"].apply(lambda x: x.replace("Very Good","2"))
df["cut"] = df["cut"].apply(lambda x: x.replace("Good","1"))
df["cut"] = df["cut"].apply(lambda x: x.replace("Ideal","3"))
df["cut"] = df["cut"].apply(lambda x: x.replace("Premium","4"))
df["cut"] = df["cut"].astype(int)

df["color"] = df["color"].apply(lambda x: x.replace("D","0"))
df["color"] = df["color"].apply(lambda x: x.replace("E","1"))
df["color"] = df["color"].apply(lambda x: x.replace("F","2"))
df["color"] = df["color"].apply(lambda x: x.replace("G","3"))
df["color"] = df["color"].apply(lambda x: x.replace("H","4"))
df["color"] = df["color"].apply(lambda x: x.replace("I","5"))
df["color"] = df["color"].apply(lambda x: x.replace("J","6"))
df["color"] = df["color"].astype(int)

df["clarity"] = df["clarity"].apply(lambda x: x.replace("SI1","4"))
df["clarity"] = df["clarity"].apply(lambda x: x.replace("SI2","5"))
df["clarity"] = df["clarity"].apply(lambda x: x.replace("I1","0"))
df["clarity"] = df["clarity"].apply(lambda x: x.replace("IF","1"))
df["clarity"] = df["clarity"].apply(lambda x: x.replace("VVS1","2"))
df["clarity"] = df["clarity"].apply(lambda x: x.replace("VVS2","3"))
df["clarity"] = df["clarity"].apply(lambda x: x.replace("VS1","6"))
df["clarity"] = df["clarity"].apply(lambda x: x.replace("VS2","7"))
df["clarity"] = df["clarity"].astype(int)


In [4]:
X = np.array(df["carat"]).reshape(-1, 1)
y = np.array(df["price"])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 13)

In [7]:
svm = Pipeline(steps=[
    ("scaler", StandardScaler()),
    #("selectkbest", SelectKBest()),
    ("svm", SVR())])

svm_param = {"svm__kernel": ("linear", "rbf", "sigmoid", "precomputed")
,'svm__kernel': ('linear', 'rbf', 'sigmoid'),
    'svm__C': [700, 1000, 2000, 2200, 3000, 5000, 10000]
    'svm__gamma': ('scale', 'auto')
    'svm__epsilon': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 1, 1.5, 1.6]
    'svm__max_iter': [99999, 999999]}

gs_svm = GridSearchCV(svm, svm_param,
                         cv=10
                         scoring = 'neg_mean_squared_error',
                         verbose=1)

In [8]:
grids = {
    "gs_svm": gs_svm
}

In [9]:
for nombre, grid in grids.items():
    grid.fit(X_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
Fitting 5 folds for each of 36 candidates, totalling 180 fits


In [ ]:
import pandas as pd
best_grids = [(i, j.best_score_) for i, j in grids.items()]

best_grids = pd.DataFrame(best_grids,
                         columns = ['Grid', 'Best score']).sort_values(by = 'Best score', ascending=False)
best_grids